In [75]:
from crewai import Task , Agent, Crew
from crewai.tools import tool
   
from crewai_tools import SerperDevTool
import os
import requests
os.environ['SERPER_API_KEY'] = "42e63eb2b80d1d7d07a80de1e745303952c827c7"


@tool("search_google")
def search_google(question: str) -> str:
    """Search the internet based on the name of the person  
    """
    tool = SerperDevTool(
        search_url="https://google.serper.dev/scholar",  
        n_results=2,
    )

    result = tool.run(search_query=question)
   
    return result


@tool("get_all_restaurants")
def get_all_restaurants(location_name: str):
    """Call the get api for fetch all the restaurants in a location name
    """
    url = f"https://v2api-dev.alacater.com/v2/customers/all-restaurants/{location_name}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        print(data)
    else:
        print(f"Error: {response.status_code}")
        data = "No data found"
    
    return data






agent_one = Agent(role="You are a restaurant fething agent",
                  goal="check for fetching all the restaurant in the corresponding location",
                  backstory="if u can't find any restaurant ,tell you don't know ",
                  tools=[search_google, get_all_restaurants])

In [76]:
task_one = Task(description="Get all the restaurnat in the following location : {location}, if the  result is empty,  tell u dont know",
                expected_output="Show json format",
                agent=agent_one)

In [77]:
crew = Crew(agents=[agent_one],
            tasks=[task_one],
            verbose=True)

2025-01-13 09:50:16,384 - 8187826944 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [78]:
result = crew.kickoff(inputs={"location": "Al barsha first"})
print(result)

# Agent: You are a restaurant fething agent
## Task: Get all the restaurnat in the following location : Al barsha first, if the  result is empty,  tell u dont know
[{'_id': '6773b0b36d70d085f79309e8', 'name': 'Gonzalez y Gonzalez', 'location': 'Al Barsha First', 'address': 'Al Barsha First , UAE', 'description': 'Gonzalez y Gonzalez is a Mexican restaurant with multiple locations.', 'latitude': '25.1071127', 'longitude': '55.1984575', 'image': 'https://alacater-dev.s3.me-central-1.amazonaws.com/cafe1.webp', 'bannerImage': 'https://alacater-dev.s3.me-central-1.amazonaws.com/Taco-Titanic.webp', 'cuisine_types': ['67244b714c0195aad6f43ef0', '67244b714c0195aad6f43ef1', '67244b714c0195aad6f43ef2', '67244b714c0195aad6f43ef3', '67244b714c0195aad6f43ef5', '67244b714c0195aad6f43ef6', '67244b714c0195aad6f43efa', '67244b714c0195aad6f43efb', '67244b714c0195aad6f43efc', '67244b714c0195aad6f43efe', '67244b714c0195aad6f43eff', '67244b714c0195aad6f43f00'], 'minimum_price': 5, 'website': 'www.gonzalez-

In [80]:
agent_one.llm.model

'gpt-4o-mini'